In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from datasets import load_dataset,load_metric

In [2]:

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [3]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["[PAD]","[UNK]", "[CLS]", "[SEP]",  "[MASK]"],vocab_size=32768)

In [4]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [24]:
seed=2
max_length = 64

In [8]:

dataset = load_dataset('wmt14','de-en')
train = dataset['train'].shuffle(seed=seed)
train = train['translation']

Reusing dataset wmt14 (C:\Users\kevin\.cache\huggingface\datasets\wmt14\de-en\1.0.0\d239eaf0ff090d28da19b6bc9758e24634d84de0a1ef092f0b5c54e6f132d7e2)
100%|██████████| 3/3 [00:00<00:00, 34.98it/s]
Loading cached shuffled indices for dataset at C:\Users\kevin\.cache\huggingface\datasets\wmt14\de-en\1.0.0\d239eaf0ff090d28da19b6bc9758e24634d84de0a1ef092f0b5c54e6f132d7e2\cache-43836e47740ac86b.arrow


In [9]:

batch_size = 1000

def batch_iterator():
    for i in range(0, len(train), batch_size):
        tt = train[i : i + batch_size]
        yield [t['en']+' '+t['de'] for t in tt]

In [12]:
tokenizer.train_from_iterator(batch_iterator(),trainer)

In [13]:
tokenizer_T5 = T5Tokenizer.from_pretrained("google/t5-small-lm-adapt")

NameError: name 'T5Tokenizer' is not defined

In [58]:
tokenizer

PreTrainedTokenizer(name_or_path='google/t5-small-lm-adapt', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_

In [44]:
tokenizer.save("tokenizer-en-de.json")

AttributeError: 'PreTrainedTokenizerFast' object has no attribute 'save'

In [40]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer-en-de.json")
tokenizer.add_special_tokens({'pad_token': '[PAD]','unk_token':'[UNK]','eos_token':'[SEP]'})

In [36]:
tokenizer

In [29]:
tokenizer.pad_token = 0

In [46]:
tokenizer(['im kevin','im also kevin ah ah ah'],return_tensors='pt', padding=True, truncation = True, max_length = max_length)

{'input_ids': tensor([[3816, 3851, 8032,    0,    0,    0,    0],
        [3816, 4086, 3851, 8032, 3955, 3955, 3955]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

In [35]:
tokenizer = Tokenizer.from_file("tokenizer-en-de.json")
tokenizer.token_to_id("[PAD]")

0